<a href="https://colab.research.google.com/github/englissi/product/blob/main/(%ED%99%94%EC%83%81)Blue_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install gradio speechrecognition pydub gtts



In [14]:
import gradio as gr
import speech_recognition as sr
from pydub import AudioSegment
import os
from gtts import gTTS

def transcribe_audio(audio):
    recognizer = sr.Recognizer()
    audio = AudioSegment.from_file(audio)
    audio.export("temp.wav", format="wav")
    with sr.AudioFile("temp.wav") as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
        except sr.UnknownValueError:
            text = "Sorry, I could not understand the audio."
        except sr.RequestError:
            text = "Sorry, there was an error with the speech recognition service."
    os.remove("temp.wav")
    return text

def speak_text(text):
    tts = gTTS(text)
    tts.save("question.mp3")
    return "question.mp3"

questions = [
    {"context": "Based on the story Dinosaurs. I like dinosaurs. Triceratops are my favorite. Triceratops have three horns and they walk on 4 legs.", "question": "What do triceratops eat when they get hungry?", "label": "Transcription of triceratops' diet", "answer": "When Triceratops get hungry, they eat plants."},
    {"context": "Tyrannosaurus are big. They grow up to 50 feet.", "question": "What do tyrannosaurus rex eat when they get hungry?", "label": "Transcription of tyrannosaurus' diet", "answer": "When they get hungry, they eat other dinosaurs."},
    {"context": "How about you?", "question": "Do you like dinosaurs?", "label": "Transcription of personal preference", "answer": "No, I don’t."},
    {"context": "Based on the story enjoy your dog more. I have a dog. It keeps taking baths. The keeper said I should not put it in the bathtub.", "question": "Can you give me some tips?", "label": "Transcription of tips for dog bath", "answer": "Put his favorite toys in the bathtub."},
    {"context": "My dog often fights with other dogs.", "question": "What should I do when he fights with other dogs? Should I spray them with water or not?", "label": "Transcription of action to stop dog fights", "answer": "I should spray them with water."},
    {"context": "How about you?", "question": "Do you have a dog? Do you want to have a dog?", "label": "Transcription of personal dog ownership", "answer": "I hope that someday you get a dog."},
    {"context": "Based on the story Zoo was so much fun. Look at the pictures.", "question": "Where are the gorillas?", "label": "Transcription of gorillas' location", "answer": "They are on the trees."},
    {"context": "Look at the tigers.", "question": "Where are they? And what are they doing? Are they sleeping in the sun or under the sun?", "label": "Transcription of tigers' location and activity", "answer": "They are sleeping under the sun."},
    {"context": "Look at the hippos.", "question": "Where are they and what are they doing?", "label": "Transcription of hippos' location and activity", "answer": "They are cooling off in the water."},
    {"context": "Great job.", "question": "Do you like to go to the zoo?", "label": "Transcription of zoo preference", "answer": "Yes"},
    {"context": "Who do you go to the zoo with?", "question": "Do you go to the zoo with your family, classmates or friends?", "label": "Transcription of zoo companions", "answer": "I go to the zoo with my family."},
    {"context": "Which of these animals do you like the most? Do you like the gorillas? Tigers? Lions? Or the hippos?", "question": "Which animal do you like the most?", "label": "Transcription of favorite zoo animal", "answer": "I like lions."},
    {"context": "Based on the story Cats and dogs. Cats and dogs are different in many ways. For example, dogs play during the day.", "question": "How about cats? What do they do during the day?", "label": "Transcription of cats' daytime activity", "answer": "They are sleeping during the day."},
    {"context": "When do cats play?", "question": "Dogs wag their tails when they are happy. What about cats? When do cats move their tails?", "label": "Transcription of cats' tail movement", "answer": "Cats move their tails when they are angry."}
]

current_question = 0
responses = []

def next_question():
    global current_question
    if current_question < len(questions):
        context = questions[current_question].get("context", "")
        question = questions[current_question]["question"]
        full_text = f"{context} {question}"
        question_audio = speak_text(full_text)
        current_question += 1
        return gr.update(value=question_audio, visible=True), gr.update(visible=True), questions[current_question-1]["label"], gr.update(visible=True), gr.update(visible=False)
    else:
        final_results = evaluate_responses()
        return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value=final_results, visible=True)

def save_response(audio):
    transcription = transcribe_audio(audio)
    responses.append(transcription)
    return transcription

def evaluate_responses():
    result = "<h2>Your Responses:</h2><br>"
    for i, question in enumerate(questions):
        user_response = responses[i] if i < len(responses) else "No response"
        result += f"<b>Q:</b> {question['question']}<br><b>Your Answer:</b> {user_response}<br><br>"
    return result

with gr.Blocks() as demo:
    gr.Markdown("### Interactive Questions")

    question_audio = gr.Audio(label="Question", visible=False)
    audio_input = gr.Audio(type="filepath", label="Your answer", visible=True)
    transcription_output = gr.Textbox(label="Transcription", visible=True)
    btn_next = gr.Button("Next", visible=True)
    final_output = gr.HTML(visible=False)

    def load_first_question():
        return next_question()

    demo.load(load_first_question, outputs=[question_audio, audio_input, transcription_output, btn_next, final_output])

    btn_next.click(next_question, outputs=[question_audio, audio_input, transcription_output, btn_next, final_output])
    audio_input.change(save_response, inputs=audio_input, outputs=transcription_output)

    demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9d52b9aa6444722ea2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
